In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from mongo_crud import DatabaseHandler

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU4321"
collection = "animals"
database = "AAC"

# Connect to database via CRUD Module
db = DatabaseHandler(username, password, database, collection)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
# Static images won't work for some reason, encoding
image_filename = 'grazlogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.Div([
        html.Center(
            html.A( href="www.snhu.edu", children=[
                html.Img(style = {'width': '250px'}, src = 'data:image/png;base64,{}'.format(encoded_image.decode()))
            ]))
    ]),
    html.Center(html.B(html.H1('CS-340 Dashboard - Brooklen Ashleigh'))),
    html.Hr(),
    html.H3("Select for Rescue Type"),
    html.Div(
        dcc.RadioItems(['Water Rescue', 'Mountain or Wilderness Rescue', 'Disaster or Individual Tracking', "Reset"], id = "sortRadio")
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10,
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-content',
            style={"width":"50vw"},
            className='col s12 m6'),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id','data'),
              [Input('sortRadio', 'value')])
def update_dashboard(radioValue):
          
    df = pd.DataFrame.from_records(db.read({}))

    # Radio Button Sort Queries
    # Water Rescue
    water_rescue_query = {
        "$and" : [
            {"animal_type": "Dog"},
            {
                "$or" : [
                    {"breed": {'$regex':"Labrador Retriever Mix"}},
                    {"breed": {'$regex':"Chesa Bay Retr"}}, 
                    {"breed": {'$regex':"Newfoundland"}}
                ]
            },
            {"sex_upon_outcome": "Intact Female"},
            {"age_upon_outcome_in_weeks": {"$lte":156}},
            {"age_upon_outcome_in_weeks": {"$gte":26}}
        ]
    }
    
    #  Mountain and Wilderness Rescue
    mw_rescue_query = {
        "$and" : [
            {"animal_type": "Dog"},
            {
                "$or" : [
                    {"breed": {'$regex':"German Shepherd"}},
                    {"breed": {'$regex':"Alaskan Malamute"}}, 
                    {"breed": {'$regex':"Old English Sheepdog"}},
                    {"breed": {'$regex':"Siberian Husky"}}, 
                    {"breed": {'$regex':"Rottweiler"}},
                ]
            },
            {"sex_upon_outcome": "Intact Male"},
            {"age_upon_outcome_in_weeks": {"$lte":156}},
            {"age_upon_outcome_in_weeks": {"$gte":26}}
        ]
    }
    
    #  Disaster or Individual Tracking
    di_rescue_query = {
        "$and" : [
            {"animal_type": "Dog"},
            {
                "$or" : [
                    {"breed": {'$regex':"Doberman Pinsch"}},
                    {"breed": {'$regex':"German Shepherd"}}, 
                    {"breed": {'$regex':"Golden Retriever"}},
                    {"breed": {'$regex':"Bloodhound"}}, 
                    {"breed": {'$regex':"Rottweiler"}},
                ]
            },
            {"sex_upon_outcome": "Intact Male"},
            {"age_upon_outcome_in_weeks": {"$lte":300}},
            {"age_upon_outcome_in_weeks": {"$gte":20}}
        ]
    }
    
    # Choose query depending on Radio button
    if radioValue == "Water Rescue":
        df = pd.DataFrame.from_records(db.read(water_rescue_query))
    elif radioValue == "Mountain or Wilderness Rescue":
        df = pd.DataFrame.from_records(db.read(mw_rescue_query))
    elif radioValue == "Disaster or Individual Tracking":
        df = pd.DataFrame.from_records(db.read(di_rescue_query))
    
    # Cleanup Mongo _id field
    df.drop(columns=['_id'],inplace=True)
    
    if len(df) == 0:
        return None
    
    return df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-content', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")
    ])
def update_graphs(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData)
    
    # If nothing, return
    if len(dff) == 0:
        return
    
    # Create a second dataFrame sorted by most occurrences
    newdff = dff.groupby(['breed'])['animal_type'].count().reset_index(name='Count').sort_values(['Count'], ascending=False)
    # Slice only top 20 results for pie chart, otherwise chart is overloaded
    newdff = newdff.head(20)

    # If there are more than 30 items, deliver truncated dataframe
    if len(dff) > 30:
        return [
            dcc.Graph(figure = px.pie(newdff, names="breed", values='Count', title="20 Highest Occurrences by Breed"))
        ]
    else:
        return [
            dcc.Graph(figure = px.pie(newdff, names="breed", values='Count', title="Preferred Breeds (By Number of Occurrences)"))
        ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '50vw', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:20099/
